# Calculadora de dimensiones a partir de coordenadas almacenadas en .txt

In [1]:
import os
import numpy as np
import random

def leer_coordenadas(ruta_txt):
    """
    Lee las coordenadas desde un archivo .txt y las devuelve como un array de NumPy.
    
    Args:
        ruta_txt (str): Ruta al archivo de coordenadas .txt.
        
    Returns:
        np.ndarray: Array de coordenadas (x, y).
    """
    coordenadas = []
    with open(ruta_txt, 'r') as f:
        for line in f:
            x, y = map(float, line.strip().split(','))
            coordenadas.append((x, y))
    return np.array(coordenadas)

def calcular_dimension_correlation_incremental(coordenadas, radio_min=0.1, radio_max=10.0, num_radios=100, num_pares=100000):
    """
    Calcula la dimensión de correlación usando el método de Grassberger-Procaccia con muestreo incremental.
    
    Args:
        coordenadas (np.ndarray): Array de coordenadas (x, y).
        radio_min (float): Radio mínimo.
        radio_max (float): Radio máximo.
        num_radios (int): Número de radios a considerar entre el mínimo y el máximo.
        num_pares (int): Número de pares de puntos a muestrear aleatoriamente.
        
    Returns:
        float: Dimensión de correlación estimada.
    """
    radios = np.logspace(np.log10(radio_min), np.log10(radio_max), num=num_radios)
    conteos = np.zeros(num_radios)

    for _ in range(num_pares):
        idx1, idx2 = random.sample(range(len(coordenadas)), 2)
        distancia = np.linalg.norm(coordenadas[idx1] - coordenadas[idx2])
        
        for i, radio in enumerate(radios):
            if distancia < radio:
                conteos[i] += 1
                break

    conteos = conteos / num_pares
    log_radios = np.log(radios)
    log_conteos = np.log(conteos + 1e-10)  # Avoid log(0)

    coef = np.polyfit(log_radios, log_conteos, 1)
    dimension_correlation = coef[0]

    return dimension_correlation

def procesar_archivos_y_guardar_dimensiones(ruta_carpeta):
    """
    Procesa todos los archivos de coordenadas en una carpeta, calcula la dimensión de correlación
    para cada archivo y guarda los resultados en 'correlation_dimentions.tsv'.
    
    Args:
        ruta_carpeta (str): Carpeta que contiene los archivos de coordenadas.
    """
    ruta_tsv = 'correlation_dimentions.tsv'  # Nombre del archivo de salida

    # Leer IDs ya procesados
    if os.path.exists(ruta_tsv):
        with open(ruta_tsv, 'r') as f_in:
            processed_ids = {line.split('\t')[0] for line in f_in.readlines()[1:]}  # Ignorar encabezado
    else:
        processed_ids = set()

    with open(ruta_tsv, 'a') as f_out:  # Abrir en modo 'append' para no sobrescribir
        if not processed_ids:  # Escribir encabezado solo si el archivo está vacío
            f_out.write("ID\tCorrelation_Dimension\n")

        for filename in os.listdir(ruta_carpeta):
            if filename.endswith('.txt'):
                sample_id = os.path.splitext(filename)[0]

                # Saltar si ya ha sido procesado
                if sample_id in processed_ids:
                    print(f"Saltado: {sample_id}, ya procesado.")
                    continue
                
                print(f"Procesando: {sample_id}")
                ruta_txt = os.path.join(ruta_carpeta, filename)
                coordenadas = leer_coordenadas(ruta_txt)

                # Saltar archivos con menos de dos coordenadas
                if len(coordenadas) < 2:
                    print(f"Ignorado: {sample_id}, datos insuficientes.")
                    continue
                
                dimension_correlation = calcular_dimension_correlation_incremental(coordenadas)
                
                # Escribir el nombre del archivo (sin extensión) y su dimensión de correlación en el archivo .tsv
                f_out.write(f"{sample_id}\t{dimension_correlation}\n")
                print(f"Procesado: {sample_id} con dimensión de correlación {dimension_correlation}")
            else:
                print(f"Ignorado: {filename}, no es un archivo .txt válido.")



In [ ]:
%%time
# Ejemplo
path_input_folder = "/files/Mariel/Tesis_Mariel/data/Examples"
array_dimension = procesar_archivos_y_guardar_dimensiones(path_input_folder)